In [2]:
import pandas as pd
#Import DataSets

# Read the csv file
df = pd.read_csv('data_train.csv')
y_test = pd.read_csv('test_competition.csv')


Y_train = df["default.payment.next.month"]

X_train = df.drop(columns = ["default.payment.next.month"])






In [2]:
# import joblib 

# #Splitting DataFrame
# X_train = df.iloc[:20000,:]

# Y_train = X_train["default.payment.next.month"]

# X_train = X_train.drop(columns = ["default.payment.next.month"])



# y_test = df.iloc[20000:,:]

# y_test.to_csv("test_5000.csv", index=False)

# y_test = y_test.drop(columns = ["default.payment.next.month"])
# print(y_test)



# # # 
# # y_test = df.iloc[20000:,:]
# # y_test = y_test.drop(columns = ["default.payment.next.month"])



In [3]:
# #Using smote to overSample minorities in the dataset for avoiding bias
# from imblearn.over_sampling import SMOTE

# print("Smoting...")

# # Step 2: Apply SMOTE to the training data
# smote = SMOTE(random_state=42)
# X_train, Y_train = smote.fit_resample(X_train, Y_train)

# print("Data over sampled !")



In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

def bestModelFindingWithRandomForest(param_grid, scoring, X_train, Y_train, X_train_reshape=False, Y_train_reshape=False):
    """
    Find best Random Forest model using GridSearchCV
    
    Parameters:
    -----------
    param_grid : dict
        Parameter grid for GridSearchCV
    scoring : str
        Scoring metric ('roc_auc', 'accuracy', etc.)
    X_train : array-like
        Training features
    Y_train : array-like
        Training labels
    X_train_reshape : bool
        Whether to reshape X_train
    Y_train_reshape : bool
        Whether to reshape Y_train
    """
    try:
        print("Loading RandomForestClassifier...")
        model = RandomForestClassifier(random_state=100)
        
        if X_train_reshape:
            print("Reshaping X_train...")
            X_train = X_train.reshape(X_train.shape[0], -1)
            
        if Y_train_reshape:
            print("Reshaping Y_train...")
            Y_train = Y_train.reshape(Y_train.shape[0], -1)
            
        print("Loading GridSearchCV model...")
        # Using GridSearchCV with 10-fold cross-validation
        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grid,
            cv=7,
            scoring=scoring,
            n_jobs=-1,
            verbose=1
        )
        
        print("Fitting GridSearchCV model...")
        # Fit the model to find the best hyperparameters
        grid_search.fit(X_train, Y_train)
        
        print("GridSearchCV fitted")
        
        # Best parameters and score obtained
        best_params = grid_search.best_params_
        best_score = grid_search.best_score_ * 100  # In percentage
        
        print("\nBest parameters found:")
        for param, value in best_params.items():
            print(f"{param}: {value}")
        print(f"\nBest {scoring} score: {best_score:.2f}%")
        
        return grid_search.best_estimator_
        
    except Exception as e:
        print(f"{str(e)}")
        return None

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 150, 200],           # Number of trees
    'max_depth': [10, 15, 20, 25],             # Tree depth
    # 'class_weight': ['balanced']               # Class weight
}


bestModel = bestModelFindingWithRandomForest(param_grid, "roc_auc", X_train, Y_train)


Loading RandomForestClassifier...
Loading GridSearchCV model...
Fitting GridSearchCV model...
Fitting 7 folds for each of 12 candidates, totalling 84 fits
GridSearchCV fitted

Best parameters found:
max_depth: 10
n_estimators: 200

Best roc_auc score: 77.98%


In [5]:
!pip install lightgbm

In [6]:
# METHOD 2
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

def bestModelFinding(param_grid, scoring, X_train, Y_train):
    """
    Find the best LightGBM model using GridSearchCV
    
    Parameters:
    -----------
    param_grid : dict
        Parameter grid for GridSearchCV
    scoring : str
        Scoring metric ('roc_auc', 'accuracy', etc.)
    X_train : array-like
        Training features
    Y_train : array-like
        Training labels
    """
    # Base parameters for the model
    base_params = {
        'objective': 'binary',
        'metric': 'auc',
        'verbosity': -1,
        'random_state': 42
    }
    
    
    try:
        # Loading model
        print("Loading lightgbm...")
        # Initialize the LGBMClassifier with base parameters
        model = lgb.LGBMClassifier(**base_params)
        
        print("Loading GridSearch model...")
        # Configure GridSearchCV with 5-fold cross-validation
        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grid,
            cv=5,
            scoring=scoring,
            n_jobs=-1,
            verbose=1  #for progress tracking
        )
        
        print("Fitting GridSearch model...")
        # Fit the GridSearch model to find the best hyperparameters
        grid_search.fit(X_train, Y_train)
        print("GridSearch fitted")
        
        # Best parameters and best score found
        best_params = grid_search.best_params_
        best_score = grid_search.best_score_ * 100  # In percentage
        
        print("\nBest parameters found:")
        for param, value in best_params.items():
            print(f"{param}: {value}")
        print(f"\nBest {scoring} score: {best_score:.2f}%")
        
        return grid_search.best_estimator_
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

# Example usage:

# Define your parameter grid
param_grid = {
    'n_estimators': [600,900],
    'learning_rate': [0.01, 0.08 ,0.05],
    'max_depth': [5,10, 25],
    'num_leaves': [63 ,100,120]
}

# param_grid = { 78.21%
#     'n_estimators': [100, 200, 300],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'max_depth': [-1, 3, 5, 7],
#     'num_leaves': [31, 63, 127],
#     'min_child_samples': [20, 50],
#     'subsample': [0.8, 1.0],
#     'colsample_bytree': [0.8, 1.0]
# }

# Find the best model
bestModel = bestModelFinding(param_grid, "roc_auc", X_train, Y_train)


Loading lightgbm...
Loading GridSearch model...
Fitting GridSearch model...
Fitting 5 folds for each of 54 candidates, totalling 270 fits
GridSearch fitted

Best parameters found:
learning_rate: 0.01
max_depth: 5
n_estimators: 600
num_leaves: 63

Best roc_auc score: 78.25%


In [17]:
import pandas as pd 

# Prediction step
print("Predicting...")
predicted_survival = bestModel.predict_proba(y_test)  

# Extracting the first column of predicted probabilities
unsortedPredictedPaymentValue = predicted_survival[:, 1]

# Combining Passenger ID and predicted payment probabilities into a DataFrame
predicted_df = pd.DataFrame({
    'ID': y_test['ID'],      # Get the PassengerId from y_test
    'PAYED': unsortedPredictedPaymentValue  # Add the predicted payment probability (class 0)
})

# Sort predicted_df by the 'PAYED' column in descending order and extract sorted IDs
sorted_df = predicted_df.sort_values(by='PAYED', ascending=False)[['ID']]

# Display the resulting DataFrame of sorted IDs

sorted_df.head(1000).to_csv("sorted_df.csv", index=False)



Predicting...
